In [ ]:
%pip install datasets transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = "./drive/MyDrive/Content-Summary_CleanData.csv" 
chunksize = 10_000  # number of rows per chunk
# adjust the total number of rows to use for training (optional)
rows_to_read = 100_000


data_chunks = []
rows_loaded = 0

# reading file in chunks
for chunk in pd.read_csv(file_path, chunksize=chunksize, encoding="utf-8"):
    remaining_rows = rows_to_read - rows_loaded
    if remaining_rows <= 0:
        break

    if len(chunk) > remaining_rows:
        data_chunks.append(chunk.iloc[:remaining_rows])
    else:
        data_chunks.append(chunk)

    rows_loaded += len(chunk)

# combine chunks into a DataFrame
df = pd.concat(data_chunks, ignore_index=True)

print(f"Loaded {len(df)} rows into memory.")

Loaded 100000 rows into memory.


In [ ]:
from transformers import BartTokenizer

# Initialize the tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Preprocess function for tokenization


def preprocess_chunk(chunk):
    inputs = chunk["Content"].tolist()
    targets = chunk["Summary"].tolist()

    # tokenize inputs and targets
    model_inputs = tokenizer(inputs, max_length=1024,
                             truncation=True, padding=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128,
                           truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# tokenize each chunk
tokenized_data = []
for i, chunk in enumerate(data_chunks):
    print(f"Processing chunk {i + 1}...")
    tokenized_chunk = preprocess_chunk(chunk)
    tokenized_data.append(tokenized_chunk)

print("Tokenization complete.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Processing chunk 1...


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Processing chunk 2...
Processing chunk 3...
Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Tokenization complete.


In [ ]:
from datasets import Dataset

# convert each tokenized chunk to dataset and save
for i, tokenized_chunk in enumerate(tokenized_data):
    print(f"Converting chunk {i + 1} to Dataset format...")
    chunk_dataset = Dataset.from_dict(tokenized_chunk)

    # save each chunk to disk
    chunk_dataset.save_to_disk(f"tokenized_chunk_{i + 1}")
    print(f"Saved tokenized_chunk_{i + 1} to disk.")

Converting chunk 1 to Dataset format...


Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saved tokenized_chunk_1 to disk.
Converting chunk 2 to Dataset format...


Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saved tokenized_chunk_2 to disk.
Converting chunk 3 to Dataset format...


...

Saved tokenized_chunk_8 to disk.
Converting chunk 9 to Dataset format...


Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saved tokenized_chunk_9 to disk.
Converting chunk 10 to Dataset format...


Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saved tokenized_chunk_10 to disk.


In [ ]:
from datasets import load_from_disk, concatenate_datasets

# load tokenized chunks one by one
train_chunks = []
for i in range(len(tokenized_data)):  # adjust this loop based on the number of saved chunks
    print(f"Loading tokenized_chunk_{i + 1}...")
    chunk = load_from_disk(f"tokenized_chunk_{i + 1}")
    train_chunks.append(chunk)

# combine all chunks into a single training dataset
train_dataset = concatenate_datasets(train_chunks)

print("All tokenized chunks loaded and combined into a single training dataset.")

Loading tokenized_chunk_1...
Loading tokenized_chunk_2...
Loading tokenized_chunk_3...
Loading tokenized_chunk_4...
Loading tokenized_chunk_5...
Loading tokenized_chunk_6...
Loading tokenized_chunk_7...
Loading tokenized_chunk_8...
Loading tokenized_chunk_9...
Loading tokenized_chunk_10...
All tokenized chunks loaded and combined into a single training dataset.


In [ ]:
from transformers import BartForConditionalGeneration, TrainingArguments, Trainer

# load the BART model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# remove the 'predict_with_generate' argument
training_args = TrainingArguments(
    output_dir="./results1",
    evaluation_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # No accumulation
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs1",
    save_strategy="epoch",
    fp16=True,
    dataloader_num_workers=4,
    report_to = "none"
)



def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(
        predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    return {}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# training the modell woohooo
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-6a1163ffe2f0>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Step,Training Loss
500,1.641500
1000,1.114600
1500,1.099500
2000,1.100400
2500,1.050200
3000,1.061900
3500,1.033000
4000,1.047800
4500,1.029300
5000,1.035700


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=12500, training_loss=1.0453615844726563, metrics={'train_runtime': 4821.5022, 'train_samples_per_second': 20.74, 'train_steps_per_second': 2.593, 'total_flos': 6.0973645824e+16, 'train_loss': 1.0453615844726563, 'epoch': 1.0})

In [ ]:
# # Evaluate the model
# metrics = trainer.evaluate()
# print("Evaluation Metrics:", metrics)

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_bart")
tokenizer.save_pretrained("./fine_tuned_bart")

('./fine_tuned_bart/tokenizer_config.json',
 './fine_tuned_bart/special_tokens_map.json',
 './fine_tuned_bart/vocab.json',
 './fine_tuned_bart/merges.txt',
 './fine_tuned_bart/added_tokens.json')

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_bart")
tokenizer.save_pretrained("./fine_tuned_bart")

print("Model and tokenizer saved to './fine_tuned_bart'.")


Model and tokenizer saved to './fine_tuned_bart'.


In [ ]:
import shutil

shutil.make_archive("fine_tuned_bart", 'zip', "./fine_tuned_bart")


'/content/fine_tuned_bart.zip'

In [ ]:
from google.colab import files
files.download("fine_tuned_bart.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>